In [1]:
from __future__ import division, print_function

In [2]:
%load_ext autoreload
%autoreload 2
%pylab inline
%load_ext wurlitzer


Populating the interactive namespace from numpy and matplotlib


In [3]:
import time
import numpy as np
from collections import namedtuple


from director import viewerclient as vc
from irispy import Polyhedron
import boxatlas.boxatlas as box
from boxatlas.contactstabilization import BoxAtlasContactStabilization

Loading cdd global constants


In [4]:
vis = vc.Visualizer()

In [5]:
surfaces = [
    box.Surface(Polyhedron.fromBounds([1.2, 0], [1.2, 2]),
            Polyhedron(np.array([[1, -1], [1, 1]]), np.array([0, 0]))),
    box.Surface(Polyhedron.fromBounds([-2, 0], [2, 0]), 
            Polyhedron(np.array([[-2, -1], [2, -1]]), np.array([0, 0]))),
    box.Surface(Polyhedron.fromBounds([-2, 0], [2, 0]), 
            Polyhedron(np.array([[-2, -1], [2, -1]]), np.array([0, 0]))),
    box.Surface(Polyhedron.fromBounds([-1.2, 0], [-1.2, 2]),
            Polyhedron(np.array([[-1, -1], [-1, 1]]), np.array([0, 0]))),
]

env = box.Environment(surfaces, Polyhedron.fromBounds([-1.2, 0], [1.2, 2]))
atlas = box.BoxAtlas()
initial_state = box.BoxAtlasState(atlas)
initial_state.qcom = np.array([0, 1])
initial_state.vcom = np.array([-4, 0.])
initial_state.qlimb = map(np.array, [[0.75, 1], [0.25, 0], [-0.25, 0], [-0.75, 1]])

opt = BoxAtlasContactStabilization(initial_state, env)
states, inputs, contact_indicator = opt.solve()
ts = states.components[0].breaks

SolutionResult.kSolutionFound
Optimize a model with 2036 rows, 680 columns and 5274 nonzeros
Model has 246 quadratic objective terms
Coefficient statistics:
  Matrix range    [2e-03, 1e+03]
  Objective range [1e+01, 6e+03]
  Bounds range    [1e+00, 2e+00]
  RHS range       [2e-01, 1e+03]
Presolve removed 599 rows and 223 columns
Presolve time: 0.01s
Presolved: 1437 rows, 457 columns, 3665 nonzeros
Presolved model has 236 quadratic objective terms
Variable types: 381 continuous, 76 integer (76 binary)

Root relaxation: objective -2.249574e+04, 1350 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -22495.741    0   76          - -22495.741      -     -    0s
     0     0 -22495.741    0   76          - -22495.741      -     -    0s
H    0     0                    -22091.47952 -22495.741  1.83%     -    0s
     0     2 -22438.224    0   67 -22091.480

In [7]:
solnData = namedtuple("SolutionData", ["opt" ,"states", "inputs", "contact_indicator", "ts"])
solnData.opt = opt
solnData.states = states
solnData.inputs = inputs
solnData.contact_indicator = contact_indicator
solnData.ts = ts

box.planPlayback(vis, solnData)

In [ ]:
[inputs(t).flimb[3] for t in ts[:-1]]

In [ ]:
[f.coeffs[0] for f in contact_indicator[3].functions]

In [ ]:
i = 0


In [ ]:
box.draw(vis, states(min(ts[i], ts[-1]-0.001)), inputs(min(ts[i], ts[-1]-0.001)), env)
if i < len(ts) - 1:
    i += 1
else:
    i = 0